In [6]:
using Cubature

In [1]:
function R(n::Int64, m::Int64, ρ::Float64)
    if (n - m) % 2 != 0
        0
    else
        function summand(k)
            n = big(n)
            k = big(k)
            (-1)^k * factorial(n-k)/(factorial(k)*factorial(Int((n+m)/2) - k)*factorial(Int((n-m)/2) - k))*(ρ)^(n-2*k)
        end
        mapreduce(summand, +, Array(range(0, stop=Int((n-m)/2), step=1)))
    end
end

function Z(n, m, ρ, θ)
    if m < 0
        R(n, abs(m), ρ) * sin(abs(m) * θ)
    else
        R(n, m, ρ) * cos(m * θ)
    end
end

Z (generic function with 1 method)

In [2]:
σ1 = .1
σ2 = .2
displacement = 0
function integrand(n, m)
    function rtn(coor)
        ρ = coor[1]
        θ = coor[2]
        x = ρ * cos(θ) + displacement
        y = ρ * sin(θ)
        Z(n, m, ρ, θ) * exp(-x^2/σ1^2 - y^2/σ2^2) * ρ
    end
    rtn
end

integrand (generic function with 1 method)

In [3]:
function neumann(m)
    if m == 0
        2
    else 
        1
    end
end
function cond_eval(n, m)
    if -n ≤ m ≤ n
        (2*n+2)/(π*neumann(m)) * hcubature(integrand(n, m), [0., 0.], [1., 2*π], maxevals=10000)[1]
    else
        0
    end
end

cond_eval (generic function with 1 method)

In [4]:
# maxn=100
# coeffs1 = [sum([c[1]^2 for c in [cond_eval(n, m) for n in range(0, maxn, step=1)]]) for m in range(0, maxn, step=1)];
# coeffs2 = [sum([c[1]^2 for c in [cond_eval(n, m) for n in range(0, maxn, step=1)]]) for m in range(-maxn, 0, step=1)];

In [5]:
using Plots
gr()
maxn = 20

heatmap(hcat([[c[1] for c in [cond_eval(n, m) for n in range(0, Int(1.5*maxn), step=1)]] for m in range(0, maxn, step=1)]...))

LoadError: UndefVarError: hcubature not defined

In [8]:
data = [[c[1] for c in [cond_eval(n, m) for n in range(0, Int(1.5*maxn), step=1)]] for m in range(0, maxn, step=1)]

31-element Array{Array{T,1} where T,1}:
 [0.08416513712460388, 0.0, -0.12232377196674751, 0.0, 0.11279824667521597, 0.0, -0.10295362836940096, 0.0, 0.09387462550324009, 0.0  …  0.0037867114445583583, 0.0, -0.0026231032866881085, 0.0, 0.0016621868641405167, 0.0, -0.0012076948423546746, 0.0, 0.0004745951519071059, 0.0]
 Real[0, 8.446218382845295e-18, 0.0, 8.92870151361642e-18, 0.0, -1.0819269668389892e-17, 0.0, 2.971782872007135e-11, 0.0, 1.553325388074586e-17  …  0.0, 3.553632563589662e-13, 0.0, -3.474849169811826e-17, 0.0, 3.2386629599456587e-18, 0.0, -2.0024725267085716e-7, 0.0, -1.2553267510889115e-13]
 Real[0, 0, -0.12513443397448154, 0.0, 0.17749362165119562, 0.0, -0.18556384764997452, 0.0, 0.17734319661290912, 0.0  …  0.007553433430362625, 0.0, -0.005178278884621901, 0.0, 0.0036286488960610174, 0.0, -0.002415533389425169, 0.0, 0.002068737130253801, 0.0]
 Real[0, 0, 0, -4.296863879493244e-17, 0.0, -4.982558399916398e-19, 0.0, 1.7549228554770478e-17, 0.0, 3.0825696103539534e-13  …  

In [ ]:
function recon(ρ, ϕ)
    t = 0
    for (n, row) in enumerate(data)
        for (m, col) in enumerate(row)
            t += (n + 1)*col*Z(n,m,ρ)
end
